In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne

import sklearn as sk
import pandas as pd

# Data preprocessing

Here we setup the data to feed it to the classifier model.

The data is located in the folder `../data/EDF_format/`. Each file name and the class it belongs to (Y, output variable) is listed in `../data/eeg_grades.csv`.

We want to create columns (lists): file_ID or ID, grade, `fileref`*. The fileref column contains the reference to the EDF file open via `mne.io.read_raw_edf()` from which the manipulations are done. We then slice each file into multiple epochs of fixed length.

The input dataframe for the classifier model will have columns: fileID_epochNum | grade (class) | extracted features

In [2]:
### importing dataset info from eeg_grades.csv: fileID, babyID, num, grade
### we want 'fileID' and 'grade'

import csv

data_info = {}
data_classes = {}
# get data info from eeg_grades.csv
info_filepath = "../data/eeg_grades.csv"
with open(info_filepath, 'r') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)  # skip header line
    for row in reader:
        data_info[row[0]] = (row[1], row[2], row[3])
        data_classes[row[0]] = row[3]

In [3]:
# setting up data files

files_basepath = "../data/EDF_format/"
files_names_all = list(data_classes.keys())
files_names = [name for name, c in data_classes.items() if c != '']
data_files = {}
for filename in files_names:
    file = mne.io.read_raw_edf(files_basepath+filename+".edf", preload=True, verbose=False)
    data_files[filename] = (file)

In [4]:
# importing the data
# to load data into memory we need to:
raw = data_files[files_names[0]].load_data()

In [5]:
print("EDF information")
channels = raw.ch_names
print("Channels: " + ','.join(raw.ch_names))
sfreq = raw.info['sfreq']
print("Sampling frequency: " + str(raw.info['sfreq']))
print("Duration (s): " +  str(float(raw.n_times)/raw.info['sfreq']) )

EDF information
Channels: F4,C4,T4,O2,F3,C3,T3,O1,Cz
Sampling frequency: 256.0
Duration (s): 3600.0


In [6]:
# set EEG ref as Cz? https://mne.tools/stable/auto_tutorials/preprocessing/55_setting_eeg_reference.html
# need to load data to work (preload=True, or raw.load_data)
# we also drop Cz as it is now a reference
# for raw_id in files_names:
    # data_files[raw_id].set_eeg_reference(ref_channels=['Cz'], verbose=False)
    # data_files[raw_id] = data_files[raw_id].drop_channels('Cz')

channels = data_files[files_names[0]].ch_names

- Each datafile has an EEG recording (duration 1 h or 3600 s) for 8 channels (+ `Cz` as reference).
- For each datafile a class (grade from 1 to 4) is given. 
- If the file does not have a grade, it is part of the prediction set (competition), so we remove it for now, as we are just training the model.

In [7]:
# For each datafile, separate into epochs.

epoch_duration = 30 # in seconds
epoch_overlap_t = 0

data_epochs = {}

for fid, file in data_files.items():
    data_epochs[fid] = mne.make_fixed_length_epochs(
        file, duration=epoch_duration, overlap=epoch_overlap_t, verbose=False)


# Feature extraction

## PSD - Power Spectral Density

The features to be extracted will be the average power spectral density in specific frequency bins.

so now we build a pandas dataframe in the shape:

fileID_epochID (index) | class (y) | PSD_Ch1 (shape = array) | ... | PSD_Ch9 (array)  (9 features of np.array)

OR

fileID_epochID (index) | class (y) | PSD_Ch1_F1 (shape = float) | PSD_Ch1_F2 | ... | PSD_Ch9_F38 (float)  (38*9 = 342 features)

I will try the second method:

In [8]:
# 30s @ 200 Hz
epoch_len = 30*200

In [9]:
# freq bins for aggregation
freqs = np.geomspace(0.1, 30.0, num=9)
freq_bins = [(freqs[i], freqs[i+1]) for i in range(len(freqs)-1)]
freq_bins
n_points_fft = 2048 

In [10]:
cols = []

for i in range(len(freq_bins)):
    for ch_name in channels:
        cols.append('PSD_{:s}_f{:d}'.format(ch_name, i))


In [11]:
df = pd.DataFrame(columns=(cols+['grade']))

fids = [files_names[i] for i in range(3)]

for fid, epochs in data_epochs.items():
# for fid in fids:
    # epochs = data_epochs[fid]
    epochs.drop_bad(verbose=False)
    n_epochs = len(epochs)
    epochs_psds, epochs_psds_freqs = mne.time_frequency.psd_welch(
        epochs, fmin=0.1, fmax=30.0, n_fft=n_points_fft, verbose=False)
    epochs_psds /= np.sum(epochs_psds, axis=-1, keepdims=True)
    X = []
    for (fmin,fmax) in freq_bins:
        psds_bands = epochs_psds[:, :, (epochs_psds_freqs >= fmin) & (epochs_psds_freqs < fmax)].mean(axis=-1)
        X.append(psds_bands.reshape(len(epochs_psds), -1))
    X = np.concatenate(X, axis=1)
    epochs_features = pd.DataFrame(X, columns=cols, dtype=np.dtype(float))
    epochs_features['grade'] = [data_classes[fid]]*n_epochs

    df = pd.concat([df, epochs_features], ignore_index=True)
    

c:\Python37\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [12]:
df[cols] = df[cols].astype('float')
df['grade'] = df['grade'].astype('category')

In [13]:
df.describe()

,PSD_F4_f0,PSD_C4_f0,PSD_T4_f0,PSD_O2_f0,PSD_F3_f0,PSD_C3_f0,PSD_T3_f0,PSD_O1_f0,PSD_Cz_f0,PSD_F4_f1,...,PSD_Cz_f6,PSD_F4_f7,PSD_C4_f7,PSD_T4_f7,PSD_O2_f7,PSD_F3_f7,PSD_C3_f7,PSD_T3_f7,PSD_O1_f7,PSD_Cz_f7
count,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,12569.000000,...,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04,1.256900e+04
mean,0.323152,0.329098,0.316987,0.318852,0.326845,0.336858,0.314173,0.328698,0.326897,0.127266,...,2.196260e-04,2.068503e-04,1.183316e-04,8.826792e-05,1.054856e-04,1.636815e-04,9.549801e-05,1.600020e-04,1.352673e-04,5.970503e-05
std,0.207294,0.208954,0.208675,0.204949,0.203157,0.208338,0.208243,0.208890,0.207218,0.066548,...,2.914535e-04,4.342999e-04,2.504480e-04,1.987508e-04,3.374096e-04,3.856515e-04,2.133908e-04,4.014707e-04,3.270955e-04,1.401175e-04
min,0.001486,0.003281,0.001121,0.002183,0.003178,0.003615,0.002845,0.001135,0.003115,0.002955,...,1.999945e-07,3.522000e-08,4.288722e-08,4.606564e-08,2.392012e-08,8.628437e-08,3.710131e-08,4.910864e-08,3.721586e-08,2.824243e-08
25%,0.153062,0.160833,0.152843,0.156984,0.162471,0.169128,0.149956,0.163115,0.160610,0.076858,...,5.343705e-05,1.422235e-05,1.135387e-05,1.351561e-05,1.139309e-05,1.409709e-05,1.118693e-05,1.554807e-05,1.188231e-05,8.736179e-06
50%,0.281674,0.284833,0.265713,0.273054,0.289546,0.297018,0.267227,0.287028,0.284308,0.116896,...,1.539860e-04,4.832792e-05,3.004191e-05,3.498999e-05,3.117250e-05,3.580749e-05,3.120121e-05,4.541531e-05,3.682110e-05,2.136704e-05
75%,0.463751,0.468046,0.445826,0.450249,0.463896,0.475233,0.448488,0.461560,0.461103,0.167477,...,2.842402e-04,1.951739e-04,8.691447e-05,7.285828e-05,6.686483e-05,1.074265e-04,7.518804e-05,1.061626e-04,8.855059e-05,4.698250e-05
max,0.968255,0.968002,0.964592,0.965068,0.974808,0.964966,0.968262,0.971932,0.955733,0.432270,...,6.424147e-03,4.609045e-03,3.868459e-03,2.896981e-03,4.815132e-03,4.759098e-03,2.972491e-03,4.737876e-03,4.155286e-03,2.978245e-03


## Entropy?

# Classifiers

In [14]:
data_df = df.dropna()
print(len(df.index))
print(len(data_df.index))

12600
12569


In [15]:
y = data_df['grade']
X = data_df[cols]

y

0        1
1        1
2        1
3        1
4        1
        ..
12595    2
12596    2
12597    2
12598    2
12599    2
Name: grade, Length: 12569, dtype: category
Categories (4, object): ['1', '2', '3', '4']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1 - SVM

In [17]:
from sklearn import svm

clf_svm = svm.SVC().fit(X_train, y_train)
clf_svm.score(X_test, y_test)

0.6607000795544948

## 2 - Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier().fit(X_train, y_train)
clf_rf.score(X_test, y_test)

0.8122513922036595

## 3 - NN

In [19]:
from sklearn.neural_network import MLPClassifier

clf_nn = MLPClassifier(hidden_layer_sizes=(150,100,50),max_iter=500, early_stopping=True).fit(X_train, y_train)
clf_nn.score(X_test, y_test)

c:\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Python37\lib\site-packages\

0.7319013524264121

## Next:

- Confusion matrix, precision-recall, and other evaluation metrics
- Cross validation: https://scikit-learn.org/stable/modules/cross_validation.html
- More feature engineering?

different classifiers: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py